<a href="https://colab.research.google.com/github/dxda6216/Q10_Temp_vs_Activity/blob/main/Q10_temp_vs_activity_inputting_data_from_Excel_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### This is a simple script to calculate Q10 values for activity.
### length by using SciPy Optimize non-linear least squares fit on Colab.
### Copyright (c) 2022 by dxda6216 (dxda6216 AT gmail DOT com)
###
#@title Q10 calculator (Temperature vs. Reaction Rate)
import numpy as np
import statistics
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import pandas as pd
from google.colab import files

# Clean up previous uploads
!rm -f *.xlsx *.csv *.dat *.zip

# Upload data file
uploaded = files.upload()
data_file = next(iter(uploaded))
tadf = pd.read_excel(data_file, header=None, index_col=None)
num_columns = len(tadf.columns)

print(tadf, '\n')
print('Number of dataset: ', str(int(num_columns/2)), '\n')

#@markdown This is to caluculate Q10 temperature coefficient for reaction rate, enzymatic activity, frequency, etc (not for circadian period).

#@markdown For the circadian period, please use https://github.com/dxda6216/q10

#@markdown Prepare an Excel file containing multiple datasets.
#@markdown - To the 1st column (Column A), enter temperature data of the 1st dataset
#@markdown - To the 2nd column (Column B), enter activity data of the 1st dataset
#@markdown - To the 3rd column (Column C), enter temperature data of the 2nd dataset
#@markdown - To the 4th column (Column D), enter activity data of the 2nd dataset
#@markdown - ⇓
#@markdown - To the *2n-1* th column, enter temperature data of the *n* th dataset
#@markdown - To the *2n* th column, enter activity data of the *n* th dataset

#@markdown - No Index (data number, name, etc.) on the 1st row (Row 1) of the Excel sheet.

#@markdown ---

# Base temperature
#@markdown Set the base temperature
Select_base_temperature = "30°C" #@param ["0\u00B0C", "4\u00B0C", "10\u00B0C", "15\u00B0C", "20\u00B0C", "25\u00B0C", "30\u00B0C", "35\u00B0C", "37\u00B0C", "40\u00B0C", "45\u00B0C", "50\u00B0C", "60\u00B0C", "100\u00B0C", "Minimum", "Maximum", "Average", "Median", "Set \"Base_temperature\" by slider below"]
Base_temperature = 30 # @param {type:"slider", min:0, max:100, step:0.1}

#@markdown ---

for dnum in range(0, num_columns, 2):
    x = tadf[dnum].dropna().to_numpy()
    y = tadf[dnum+1].dropna().to_numpy()
    dataname = '# ' + str(int((dnum+2)/2))
    print('Dataset ', dataname)

    print('Temperatures: ', x)
    print('Activities: ', y)

    xy_data = {'Temperature': x, 'Activity': y}
    df = pd.DataFrame(xy_data)

    maximum_x = np.max(x)
    minimum_x = np.min(x)
    average_x = statistics.mean(x)
    median_x = statistics.median(x)

    base_temp_map = {
        "0\u00B0C": 0.000,
        "4\u00B0C": 4.000,
        "10\u00B0C": 10.000,
        "15\u00B0C": 15.000,
        "20\u00B0C": 20.000,
        "25\u00B0C": 25.000,
        "30\u00B0C": 30.000,
        "35\u00B0C": 35.000,
        "37\u00B0C": 37.000,
        "40\u00B0C": 40.000,
        "45\u00B0C": 45.000,
        "50\u00B0C": 50.000,
        "60\u00B0C": 60.000,
        "100\u00B0C": 100.000,
        "Minimum": minimum_x,
        "Maximum": maximum_x,
        "Average": average_x,
        "Median": median_x,
        "Set \"Base_temperature\" by slider below": Base_temperature
    }

    base_x = base_temp_map.get(Select_base_temperature)

    # Initial values for the fitting parameters
    Initial_fitting_parameter_Q10 = 1.000

    # Finding the experimental temperature that is the closest to the base
    # temperature and the experimental activity at that temperature.
    # Setting the initial fitting parameter Activity at Base Temperature to
    # that activity value.
    closest_x_value_to_bt = min(x, key=lambda z: abs(z - base_x))
    df2 = df[df['Temperature'].isin([closest_x_value_to_bt])]
    Initial_fitting_parameter_activity_at_base_temperature = df2['Activity'].mean()

    p0 = np.array([Initial_fitting_parameter_activity_at_base_temperature, Initial_fitting_parameter_Q10])

    print('\n', df, '\n')
    print('Base temperatures =', base_x,'\u00B0C\n')

    # Defining an equation for curve fitting
    # fitting parameters:
    #     rate_bt : activity at the base temperatures
    #     q10 : temperature coefficient (Q10)
    def func(x, rate_bt, q10):
        return ( q10 ** ( ( x - base_x ) * 0.1 ) ) * rate_bt

    # Figure graphical_parameter settings
    psx = [4, 4, 27, 45, 28, 45, 2, 14, 45, 14, 45, 2, 28, 43, 30]

    # Fitting the data to the defined equation
    popt, pcov = curve_fit(func, x, y, p0)

    residuals = y - func(x, *popt)
    ss_residuals = np.sum(residuals**2)
    ss_total = np.sum((y-np.mean(y))**2)
    r_squared = 1 - ( ss_residuals / ss_total )

    # Printing the results
    print(u'Estimated activity at', base_x, '\u00B0C =', '{:.3f}'.format(popt[0]), u'\u00B1', '{:.3f}'.format(pcov[0,0]**0.5))
    print(u'Q10 (temperature coefficient) =', '{:.3f}'.format(popt[1]), u'\u00B1', '{:.3f}'.format(pcov[1,1]**0.5))
    print(u'R\u00B2 =', '{:.6f}'.format(r_squared), u'\n')

    # Plotting data and fitted curve
    psx[14] = 30
    Marker_color = "red"
    Fitted_curve_line_color = "lightgreen"
    X_axis_caption = "Temperature (\u00B0C)"
    Y_axis_caption = "Activity"
    fig = plt.figure(figsize = (6,4.5))
    fcxmin = int( min(x) * 1.25 - max(x) * 0.25 )
    fcxmax = int( max(x) * 1.25 - min(x) * 0.25 ) + 1
    fcx = np.linspace(fcxmin, fcxmax, 200)

    plt.scatter(x, y, s=psx[14], color = Marker_color, label ='data')
    fcy = func(fcx, popt[0], popt[1])
    plt.plot(fcx, fcy, '--', color=Fitted_curve_line_color, label = 'Fit line    Q10 = %5.3f' %popt[1])
    plt.title(dataname)
    plt.xlabel(X_axis_caption)
    plt.ylabel(Y_axis_caption)

    plt.legend()
    plt.show()